# オブジェクト検出モデルの基本構造を学ぶ  

ここではPyTorchのYolo-v5sモデルをOpenVINOで実行する手順について学びます。

## 必要なライブラリのインポート

In [ ]:
import cv2
import numpy as np

import torch

import openvino as ov

from PIL import Image
from IPython.display import display

## Yolo-v5sモデルをダウンロードする  

ダウンロード完了後、`.eval()` APIで推論モードに設定する。

In [ ]:
pt_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
pt_model.eval()

## PyTorchのモデルをOpenVINO IRモデルに変換する  

PyTorchのモデル変換時にはモデルのシェイプ推論のために`example_input`が必要になる場合があります。推論時に使用するデータと同じシェイプのダミーデータを用意し、それを渡します。データ内容はダミーデータで構いません。  

変換が終わったら`yolov5s.xml`という名前でモデルを保存しておきます。

In [ ]:
dummy_input = torch.Tensor(size=(1,3,640,640))
pt_model(dummy_input)                              # convert_model()での変換時エラーを回避するため、一度モデルを実行しておく。

ov_model = ov.convert_model(pt_model, example_input=dummy_input)
#ov_model = ov.convert_model(pt_model, example_input=dummy_input, input=[(1,3,640,640)])    # モデルの入力シェイプを固定することも可能

ov.save_model(ov_model, 'yolov5s.xml', compress_to_fp16=True)
ov_model

## 変換されたOpenVINO IRモデルを読み込む

In [ ]:
ov_model = ov.Core().read_model('yolov5s.xml')
#ov_model.reshape((1,3,640,640))                # 入力シェイプを固定することも可能
print(ov_model)
compiled_model = ov.compile_model(ov_model, device_name='GPU.0', config={'CACHE_DIR':'./cache'})
compiled_model

## 入力画像の読み込みとプリプロセス

In [ ]:
img = cv2.imread('traffic.jpg')
print(img.shape)
img = cv2.resize(img, (640, 640))
reshaped_img = img.copy()
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # BGR -> RGB
display(Image.fromarray(img))
img = np.transpose(img, (2, 0, 1))           # HWC -> CHW  (h,w,c) -> (c,h,w)
img = np.expand_dims(img, axis=0)            # CHW -> NCHW (c,h,w) -> (1,c,h,w)
img = img.astype(np.float32)
img = img / 255
print(img.shape)

## モデルを実行し、実行時間を計測する

In [ ]:
%%timeit
res = compiled_model.infer_new_request(img)

In [ ]:
res = compiled_model.infer_new_request(img)
print(res)

## ポストプロセス

Yoloモデルの推論結果は少し複雑です。出力ノードのテンソルは`[1,25200,85]`のシェイプを持っています。これは、`B, N, 85`のフォーマットで、それぞれ下記のような意味を持ちます。

- `B` - バッチサイズ
- `N` - 検出したボックス(物体)の数

それぞれの検出ボックスのデータフォーマットは [`x`, `y`, `h`, `w`, `box_score`, `class_no_1`, ..., `class_no_80`] のようになっており、それぞれ下記のような意味を持ちます。

- (`x`, `y`) - ボックスの中心座標
- `h`, `w` - ボックスの幅と高さ
- `box_score` - 検出したボックス自身のの信頼度
- `class_no_1`, ..., `class_no_80` - 1\~80までの各オブジェクトクラスID毎の推定確率

多くの検出結果は重複（複数の検出ボックスが同一のオブジェクトを検出）しているため、最終的な推論結果を得るためにはNMS (Non-Maximum Suppression)処理を行います。  

**これらの出力テンソルフォーマットも使用するモデルによって異なります。** 別のモデルを利用する場合はモデルカードなどでモデルの仕様を確認することが必要です。仮に同じ名前のモデル(Yolo-v5s)であったとしても出力テンソルフォーマットは異なることがあります。

In [ ]:
# IOU (Intersection Over Union) ボックスの重なり度を計算
def calc_iou(box0, box1):
    b0_x0, b0_y0, b0_x1, b0_y1 = box0
    b1_x0, b1_y0, b1_x1, b1_y1 = box1
    b0_area = (b0_x1 - b0_x0) * (b0_y1 - b0_y0)
    b1_area = (b1_x1 - b1_x0) * (b1_y1 - b1_y0)
    xx0 = max(b0_x0, b1_x0)
    yy0 = max(b0_y0, b1_y0)
    xx1 = min(b0_x1, b1_x1)
    yy1 = min(b0_y1, b1_y1)
    w = max(0, xx1 - xx0)
    h = max(0, yy1 - yy0)
    intersect = w * h
    union = (b0_area + b1_area - intersect)
    iou = intersect / union
    return iou

# ボックス同士が大きく重なっている場合、信頼度の高い方だけを残す処理
# Non-Maximum suppression
# 私がいい加減に実装したNMSアルゴリズムですので、もっと最適化された実装があります。
def nms(predicts, iou_threshold=0.5):
    # predicts = x0, y0, x1, y1, score, *classes[80]
    predicts = predicts[np.argsort(predicts[:,4])]   # ボックス信頼度でソート
    res = []
    while(len(predicts)>0):
        res.append(predicts[0])
        box0 = predicts[0][:4]
        remove_indices = [0]
        # ボックスの重なり度を計算し、一定以上重なっている場合は信頼度の低い方を取り除く (後で取り除くためにインデックスを記録しておく)
        for idx in range(1, len(predicts)):
            box1 = predicts[idx][:4]
            iou = calc_iou(box0, box1)
            if iou > iou_threshold:
                remove_indices.append(idx)
        for idx in remove_indices[::-1]:  # 記録しておいた削除予定ボックスを後ろから削除していく (前から削除するとインデックス番号がずれるため)
            predicts = np.delete(predicts, idx, axis=0)
    return res

## 表示用のクラスラベル  

今回のYolo v5sモデルはMS COCOデータセットで学習されているので、coco_class_labelを使用します。  
使用するモデルによってクラス名の定義は異なりますので、他のモデルを使用する場合はモデルのスペックをよく確認してください。

In [ ]:
coco_class_label = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 
                    'fire hydrant', 'street_sign', 'stop sign','parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 
                    'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 
                    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
                    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
                    'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
                    'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet',
                    'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
                    'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush',
                    'hair brush']

pascal_voc_class_label = ['Person', 'Car', 'Bicycle', 'Bus', 'Motorbike', 'Train', 'Aeroplane', 'Chair', 'Bottle', 'Dining Table', 'Potted Plant', 
                          'TV/Monitor', 'Sofa', 'Bird', 'Cat', 'Cow', 'Dog', 'Horse', 'Sheep' ]

## 検出結果のBBox (bounding box)とラベルを画像上に描画するための関数の定義

In [ ]:
def draw_bbox(img, predicts, score_limit=0.7):
    for bbox in predicts:
        x0, y0, x1, y1 = bbox[:4].astype(np.int32)
        box_score = bbox[4]
        if box_score > score_limit:
            # Bboxの描画
            cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), thickness=1)
            # クラスラベルの描画
            class_id = np.argmax(bbox[5:])
            text = coco_class_label[class_id]
            (w, h), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, fontScale=1, thickness=1)
            cv2.rectangle(img, (x0, y0), (x0 + w, y0 - h - baseline), color=(0,255,0), thickness=-1)
            cv2.putText(img, text, (x0, y0 - baseline), cv2.FONT_HERSHEY_PLAIN, fontScale=1, color=(0,0,0), thickness=1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img)
    display(pil_img)

## 推論結果の事前加工  

ボックスの座標が`[center_x, center_y, height, width]`では扱いにくいので、`[x0, y0, x1, y1]`になるようにデータを操作します。

In [ ]:
predicts = res[0][0].copy()    # (25200, 85)
# predicts = x, y, h, w, score, *classes[80]

x = predicts[:, 0].copy()
y = predicts[:, 1].copy()
h = predicts[:, 2].copy()
w = predicts[:, 3].copy()
predicts[:, 0] = x - h/2
predicts[:, 1] = y - w/2
predicts[:, 2] = x + h/2
predicts[:, 3] = y + w/2
# predicts = x0, y0, x1, y1, score, *classes[80]

## 推論結果の表示 1 - そのまま  

NMSを適用せず、検出したすべてのBboxを描画してみます。  
一つのオブジェクトに複数のBboxが重複して割り当てられているのがわかります。

In [ ]:
print('Number of bbox =', predicts.shape[0])
score_limit = 0.65

out_img = reshaped_img.copy()
draw_bbox(out_img, predicts, score_limit)

## Bbox信頼度0.65未満のデータを切り捨ててみる

25,200個のBboxに対して素直にNMSを適用すると計算コストが大きくなりすぎます。  
先に信頼度の低いBboxを除外することで計算負荷を下げます。

まだ重なっているBboxがたくさんあります。

In [ ]:
# screen out low score predicted items
score_limit = 0.65
predicts = np.array([predict for predict in predicts if predict[4]>score_limit])
print('Number of Bbox =', predicts.shape[0])

out_img = reshaped_img.copy()
draw_bbox(out_img, predicts, score_limit)

## NMSを適用して、最終結果を描画する

In [ ]:
# NMS : Non Maximum Suppression
nms_result = nms(predicts, 0.8)
print('Number of bbox =', len(nms_result))

out_img = reshaped_img.copy()
draw_bbox(out_img, nms_result, score_limit)

## まとめ  

ここではYolo v5sモデルを使用した物体検出プログラムの基礎を学習しました。物体検出モデルは出力テンソルのフォーマットもモデルごとに微妙に異ります。また、モデルによってはbbox (bounding box)を直接検出するのではなく、「物体の存在確率」をヒートマップで出力するようなタイプのモデルもあります。物体検出モデルに限った話ではありませんが、モデルに合わせたポストプロセスを実装する必要があることを覚えておいてください。